<a href="https://colab.research.google.com/github/hthomas229/PurpleCrown/blob/main/maven_music.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Maven Music Challenge Pandas Data Cleaning

##Install the necessary packages.

In [24]:
import pandas as pd

#!pip install pandas

##Get the data from a .csv  file   
*Data is available for download here:*  https://mavenanalytics.io/challenges/maven-music-challenge/e161353d-9967-4297-869c-505de168e610

I dowloaded and used the file upload in the Colab Files pane to the left and then rt clk on the file and copied the path.

In [2]:
df =pd.read_csv('/content/spotify_history.csv') #paste your path here

##Investigate the Data

In [25]:
df.head()  # display first 5 rows of the dataframe

,spotify_track_uri,ts,platform,ms_played,track_name,artist_name,album_name,reason_start,reason_end,shuffle,skipped,genre
0,6WuufZDBNoEJBTinRiznhO,2018-01-01 19:24:40,android,3833,La Nueva Y La Ex,Daddy Yankee,King Daddy,clickrow,fwdbtn,True,False,"latin hip hop, reggaeton, trap latino, urbano ..."
1,54GBh4TNwKdj77GjMLjLH9,2018-01-01 19:25:00,android,19489,Plakito (feat. El General Gadiel & Farruko) - ...,Yandel,Plakito (feat. El General Gadiel & Farruko),fwdbtn,fwdbtn,True,False,"electro latino, reggaeton, trap latino, urbano..."
2,2FqoR7diymD1Eh7cVb3DMg,2018-01-01 19:29:35,android,274925,Algo Me Gusta De Ti,Wisin & Yandel,Líderes,fwdbtn,trackdone,True,False,"electro latino, reggaeton, trap latino, urbano..."
3,10YlQuxblNigpSAi8BrylN,2018-01-01 19:32:36,android,180639,Picky,Joey Montana,Picky,trackdone,trackdone,True,False,"latin pop, reggaeton, urbano latino"
4,3umS4y3uQDkqekNjVpiRUs,2018-01-01 19:36:16,android,219506,El Amante,Nicky Jam,Fénix,trackdone,trackdone,True,False,"latin hip hop, reggaeton, trap latino, urbano ..."


In [5]:
df.shape

(149860, 11)

In [27]:
df.info()  #get basic info and datatypes -- notice missing values in reason columns we will deal with these later


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14817 entries, 0 to 14816
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   spotify_track_uri  14817 non-null  object
 1   ts                 14817 non-null  object
 2   platform           14817 non-null  object
 3   ms_played          14817 non-null  int64 
 4   track_name         14817 non-null  object
 5   artist_name        14817 non-null  object
 6   album_name         14817 non-null  object
 7   reason_start       14817 non-null  object
 8   reason_end         14817 non-null  object
 9   shuffle            14817 non-null  bool  
 10  skipped            14817 non-null  bool  
 11  genre              14817 non-null  object
dtypes: bool(2), int64(1), object(9)
memory usage: 1.2+ MB


In [ ]:
df.describe()  #get basic stats on numerical columns

##Change ts to datetime datatype

In [ ]:
df['ts'] = pd.to_datetime(df['ts'])

df.info()

In [ ]:
df.shape, df.head()

##Drop NAs

In [10]:
# prompt: drop nas

df = df.dropna()

In [ ]:
df.shape

##Restrict to Year 2018 Only

In [ ]:
# prompt: subset df to dates in 2018 only

df = df[df['ts'].dt.year == 2018]



In [21]:
df.shape

(14817, 12)

In [15]:
# prompt: write to spotify_history_clean.csv

# Assuming 'df' is the DataFrame created in the previous code.
# If not, uncomment and run the previous code first

# import pandas as pd
# df =pd.read_csv('/content/spotify_history.csv')
# df['ts'] = pd.to_datetime(df['ts'])
# df = df.dropna()

df.to_csv('spotify_history_clean.csv', index=False)

##Add Genre Column from Spotify API

Go to the spotify developer app: [link text](https://developer.spotify.com/).

Click on your user id (long string) in the upper right.

Go to Dashboard

Choose create app to get a Client ID and Secret.

Insert your id, secret and Spotify History file path in the code below.

In [16]:
!pip install spotipy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 7.9 MB/s eta 0:00:00


In [19]:
#Get genre for each artist and append to each row in the database
# Add a genre column

from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials
import time


# Spotify API credentials
SPOTIFY_CLIENT_ID = "XXXXXXXXXXXXXXXXXXXX"  #Replace with your client id
SPOTIFY_CLIENT_SECRET = "XXXXXXXXXXXXXXXXX" #Replace with your secret

# Initialize Spotify client
client_credentials_manager = SpotifyClientCredentials(
    client_id=SPOTIFY_CLIENT_ID,
    client_secret=SPOTIFY_CLIENT_SECRET
)
sp = Spotify(client_credentials_manager=client_credentials_manager)

# Function to fetch genre with retry mechanism
def get_genre_with_retry(artist_name):
    while True:
        try:
            # Search for the artist
            results = sp.search(q=f"artist:{artist_name}", type="artist", limit=1)
            if results['artists']['items']:
                # Return the first artist's genres
                return results['artists']['items'][0].get('genres', [])
            else:
                return None  # Artist not found
        except spotipy.exceptions.SpotifyException as e:
            if e.http_status == 429:  # Rate limit error
                retry_after = int(e.headers.get("Retry-After", 1))
                print(f"Rate limit hit. Retrying after {retry_after} seconds.")
                time.sleep(retry_after)
            else:
                print(f"An error occurred: {e}")
                return None

# Read the CSV file
file_path = "/content/spotify_history_clean.csv"  # Replace with your file path
df = pd.read_csv(file_path)

# Create a dictionary to store unique artist genres
artist_genres = {}

# Add a new 'genre' column
genres = []
for index, row in df.iterrows():
    artist_name = row['artist_name']
    if artist_name not in artist_genres:
        print(f"Fetching genres for artist: {artist_name}")
        genre = get_genre_with_retry(artist_name)
        artist_genres[artist_name] = ", ".join(genre) if genre else "Unknown"
    genres.append(artist_genres[artist_name])

# Add the genres to the DataFrame
df['genre'] = genres

# Save the updated DataFrame to a new CSV file
output_file = "output_with_genres.csv"
df.to_csv(output_file, index=False)

print(f"Genres added to the CSV. Saved to {output_file}.")

Fetching genres for artist: Daddy Yankee
Fetching genres for artist: Yandel
Fetching genres for artist: Wisin & Yandel
Fetching genres for artist: Joey Montana
Fetching genres for artist: Nicky Jam
Fetching genres for artist: Juan Magán
Fetching genres for artist: Wisin
Fetching genres for artist: Ricky Martin
Fetching genres for artist: J Balvin
Fetching genres for artist: Don Omar
Fetching genres for artist: La Factoria
Fetching genres for artist: Kike Santander
Fetching genres for artist: The Mud Howlers
Fetching genres for artist: The Velvet Underground
Fetching genres for artist: Passenger
Fetching genres for artist: Alejandro Fernández
Fetching genres for artist: Alex Ubago
Fetching genres for artist: One Direction
Fetching genres for artist: Camila
Fetching genres for artist: The Killers
Fetching genres for artist: ZAYN
Fetching genres for artist: Reik
Fetching genres for artist: Timbiriche
Fetching genres for artist: Gente De Zona
Fetching genres for artist: Ed Sheeran
Fetching

Fetching genres for artist: Johnny Cash
Fetching genres for artist: Vance Joy


Fetching genres for artist: Golden Earring
Fetching genres for artist: Bastille
Fetching genres for artist: Keith Richards
Fetching genres for artist: Coldplay
Fetching genres for artist: Mumford & Sons
Fetching genres for artist: Little Richard
Fetching genres for artist: The Fray
Fetching genres for artist: Sin Bandera
Fetching genres for artist: MGMT
Fetching genres for artist: Bowling For Soup
Fetching genres for artist: Creedence Clearwater Revival
Fetching genres for artist: Billy Joel
Fetching genres for artist: Yusuf / Cat Stevens
Fetching genres for artist: J. Cole
Fetching genres for artist: Good Charlotte
Fetching genres for artist: Red Hot Chili Peppers
Fetching genres for artist: We The Kings
Fetching genres for artist: Rob Thomas
Fetching genres for artist: Ozzy Osbourne
Fetching genres for artist: Aerosmith
Fetching genres for artist: AC/DC
Fetching genres for artist: Guns N' Roses
Fetching genres for artist: Whitesnake
Fetching genres for artist: The Clash
Fetching genr

Fetching genres for artist: Martha Reeves & The Vandellas
Fetching genres for artist: Marc Anthony
Fetching genres for artist: Daryl Hall & John Oates


Fetching genres for artist: NEEDTOBREATHE
Fetching genres for artist: Carl Perkins
Fetching genres for artist: Bacilos
Fetching genres for artist: Nico & Vinz
Fetching genres for artist: Bob Marley & The Wailers
Fetching genres for artist: Tame Impala
Fetching genres for artist: Roy Orbison
Fetching genres for artist: WALK THE MOON
Fetching genres for artist: The Beach Boys
Fetching genres for artist: The Colourist
Fetching genres for artist: Noel Gallagher's High Flying Birds
Fetching genres for artist: Maná
Fetching genres for artist: Jorge Negrete
Fetching genres for artist: Blue Swede
Fetching genres for artist: Rupert Holmes
Fetching genres for artist: The Last Bison
Fetching genres for artist: Phoenix
Fetching genres for artist: Liam Gallagher
Fetching genres for artist: Iggy Pop
Fetching genres for artist: Brenda Lee
Fetching genres for artist: Hugo
Fetching genres for artist: Redbone
Fetching genres for artist: The Last Shadow Puppets
Fetching genres for artist: Fleetwood Mac
F

Fetching genres for artist: Ruggero Leoncavallo


Fetching genres for artist: George Frideric Handel
Fetching genres for artist: Foster The People
Fetching genres for artist: Portugal. The Man
Fetching genres for artist: Washed Out
Fetching genres for artist: Cherub
Fetching genres for artist: Variable D
Fetching genres for artist: Dennis Lloyd
Fetching genres for artist: Khalid
Fetching genres for artist: Leonard Bernstein
Fetching genres for artist: Booker T. & the M.G.'s
Fetching genres for artist: Dr. John
Fetching genres for artist: Joe Walsh
Fetching genres for artist: State Champs
Fetching genres for artist: Secondhand Serenade
Fetching genres for artist: Carlos Rivera
Fetching genres for artist: Quirino Mendoza y Cortés
Fetching genres for artist: Baha Men
Fetching genres for artist: Black Eyed Peas
Fetching genres for artist: Disturbed
Fetching genres for artist: Ozuna
Fetching genres for artist: The Lemonheads
Fetching genres for artist: Michelle Branch
Fetching genres for artist: Sum 41
Fetching genres for artist: Lit
Fetch

In [20]:
df.head()

,spotify_track_uri,ts,platform,ms_played,track_name,artist_name,album_name,reason_start,reason_end,shuffle,skipped,genre
0,6WuufZDBNoEJBTinRiznhO,2018-01-01 19:24:40,android,3833,La Nueva Y La Ex,Daddy Yankee,King Daddy,clickrow,fwdbtn,True,False,"latin hip hop, reggaeton, trap latino, urbano ..."
1,54GBh4TNwKdj77GjMLjLH9,2018-01-01 19:25:00,android,19489,Plakito (feat. El General Gadiel & Farruko) - ...,Yandel,Plakito (feat. El General Gadiel & Farruko),fwdbtn,fwdbtn,True,False,"electro latino, reggaeton, trap latino, urbano..."
2,2FqoR7diymD1Eh7cVb3DMg,2018-01-01 19:29:35,android,274925,Algo Me Gusta De Ti,Wisin & Yandel,Líderes,fwdbtn,trackdone,True,False,"electro latino, reggaeton, trap latino, urbano..."
3,10YlQuxblNigpSAi8BrylN,2018-01-01 19:32:36,android,180639,Picky,Joey Montana,Picky,trackdone,trackdone,True,False,"latin pop, reggaeton, urbano latino"
4,3umS4y3uQDkqekNjVpiRUs,2018-01-01 19:36:16,android,219506,El Amante,Nicky Jam,Fénix,trackdone,trackdone,True,False,"latin hip hop, reggaeton, trap latino, urbano ..."


##That should do it for the cleaning. I will have another video on DAX measuers and creating a dashboard in Power BI.

In [29]:
spotify_history_clean

NameError: name 'spotify_history_clean' is not defined